# Case study 2 - winter storms in France

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.cm as cm_mp
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs

from climada.hazard.base import Hazard
from climada.hazard import TCTracks
from climada.hazard import TropCyclone
from climada.entity import ImpactFuncSet
from climada.entity.impact_funcs.trop_cyclone import ImpfSetTropCyclone
from climada.entity.impact_funcs.storm_europe import ImpfStormEurope
from climada.engine import Impact
import climada.util.coordinates as u_coord

import functions as fct

#Define directories
home = str(Path.home())
project_path = home+'/Documents/CLIMADA/own_projects/parametric_casestudy/data/' 
figures_path = home+'/Library/CloudStorage/Dropbox/Aplicaciones/Overleaf/WP1_Parametric_paper_V2/art/'

#Plotting settings
plt.rcParams["font.family"] = "Arial"
plt.rcParams['font.size'] = '11'
cm = 1/2.54  # centimeters in inches
fig_width = 17.4 #cm
max_fig_height = 23.4 #cm
horizontal_plots_height = 4 #inch

In [ ]:
country_iso_FRA = 'FRA'
country_name_FRA  = 'france'
basin_FRA  = None
impf_id_FRA  = 1
hazard_name_FRA = 'storm_europe'
bounds_FRA  = None
steps_FRA  = 10 #payout
categories_FRA  = np.arange(40,70,10) #intensity
radius_FRA = np.array([10, 20, 30, 40, 50])*1000 #meter

In [ ]:
# Generate or load winter storms in France
winter_storms, ws_present_folder, hospitals_FRA, max_payout_FRA = fct.wrapper_haz_exp(hazard_name_FRA, country_iso_FRA, country_name_FRA, impf_id_FRA, 
                                             project_path, basin_FRA, bounds_FRA)
fct.wrapper_generate_haz_in_a_circle(winter_storms, hospitals_FRA, hazard_name_FRA, ws_present_folder, radius_FRA)

In [ ]:
#Compute impacts
impf = ImpactFuncSet()
impf.append(ImpfStormEurope.from_welker())

#Synthetic impacts 
damages_syn_FRA = Impact()
damages_syn_FRA.calc(hospitals_FRA, impf, winter_storms.select(orig=False), save_mat=True)
#Historic damages
damages_his_FRA = Impact()
damages_his_FRA.calc(hospitals_FRA, impf, winter_storms.select(orig=True), save_mat=True)

In [ ]:
#Parametric options
payf_FRA = fct.generate_payout_options(steps_FRA, categories_FRA)

payouts_syn, list_radius, list_payout_structure = fct.compute_payout(hazard_name_FRA, payf_FRA, categories_FRA, 
                                                                    radius, ws_present_folder, max_payout_FRA, impf_id_FRA, orig=False)

In [ ]:
#Optimization
RMSE_all = fct.RMSE_payouts(payouts_syn, impact_syn)
idx_min_errors = np.where(RMSE_all == np.min(RMSE_all)) 
min_radius = np.asarray(list_radius)[idx_min_errors]
min_structure = np.asarray(list_payout_structure)[idx_min_errors]
one_best = idx_min_errors[0][0]

#Payouts for historic storms
payouts_his, _, _ = fct.compute_payout(hazard_name, payout_options, categories, 
                                      radius, hazard_present_folder, insured, impf_id, orig=True)

In [ ]:
"""Fig7: Plotting parametric products and basis risk"""
fig7 = plt.figure()
ax7 = fig7.add_subplot(111)
fig7.set_figwidth(fig_width*cm)
categories_plot = np.arange(35,70,5)
ax7 = fct.plot_vul_his_FRA(hazard, categories_plot, impf, impf_id, min_structure[-1])
#plt.savefig(figures_path+'Fig7.jpeg', format='jpeg', dpi=300, bbox_inches='tight')